In [ ]:
!pip install -qU pinecone-client langchain cohere PyPDF2
!pip install openpyxl
!pip install  pandas transformers
!pip install -U langchain-community

In [2]:
import pandas as pd
import pinecone
import cohere

In [ ]:
#fais entrer le fichier _maajliss.xlsx
from google.colab import files
uploaded = files.upload()

In [ ]:
# Charger les données
file_path = "_majliss_table_data_cleaned.xlsx"
data = pd.read_excel(file_path)

In [ ]:
# Initialiser Cohere
cohere_api_key = 'cohere_key' # remplace with cohere_key (elle est recupere du cohere)
co = cohere.Client(cohere_api_key)

In [ ]:
# Fonction pour encoder les textes
def encode_text(text):
    response = co.embed(texts=[text], model='large')
    return response.embeddings[0]


In [ ]:
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
#Initialiser Pinecone
pc = Pinecone(
    api_key='Pinecone_key' # remplace with Pinecone_key (elle est recupere du Pinecone)
)

In [ ]:
# Configuration de l'index
index_name = 'chatboteasylaw'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=4096,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )


In [ ]:
# Accéder à l'index
index = pc.Index(index_name)

In [ ]:
import time
# Préparer les vecteurs
vectors = []
for idx, row in data.iterrows():
    combined_text = " ".join([str(row['المبدأ']), str(row['الموضوع']), str(row['التكييف']),
                              str(row['تاريخ القرار']), str(row['القسم']), str(row['الغرفة']),
                              str(row['رقم القرار'])])
    try:
        vector = encode_text(combined_text)
        vectors.append((str(idx), vector))
        time.sleep(1)  # Pause d'une seconde entre les requêtes pour éviter les limitations de taux
    except cohere.error.CohereAPIError as e:
        print(f"Erreur lors de l'encodage du texte à l'index {idx}: {e}")
        break

In [ ]:
# Fonction pour diviser les vecteurs en lots
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Taille de lot recommandée pour rester en dessous de 4 Mo
batch_size = 100

# Uploader les vecteurs dans Pinecone par lots
for batch in chunks(vectors, batch_size):
    index.upsert(batch)
    print(f"Lot de {len(batch)} vecteurs uploadé avec succès dans Pinecone")

print("Toutes les données ont été uploadées avec succès dans Pinecone")

Lot de 100 vecteurs uploadé avec succès dans Pinecone
Lot de 100 vecteurs uploadé avec succès dans Pinecone
Lot de 100 vecteurs uploadé avec succès dans Pinecone
Lot de 79 vecteurs uploadé avec succès dans Pinecone
Toutes les données ont été uploadées avec succès dans Pinecone


In [ ]:
vectors = []
for idx, row in data.iterrows():
    combined_text = " ".join([str(row['المبدأ']), str(row['الموضوع']), str(row['التكييف']),
                              str(row['تاريخ القرار']), str(row['القسم']), str(row['الغرفة']),
                              str(row['رقم القرار'])])
    try:
        vector = encode_text(combined_text)
        metadata = {
            'المبدأ': str(row['المبدأ']),
            'الموضوع': str(row['الموضوع']),
            'التكييف': str(row['التكييف']),
            'تاريخ القرار': str(row['تاريخ القرار']),
            'القسم': str(row['القسم']),
            'الغرفة': str(row['الغرفة']),
            'رقم القرار': str(row['رقم القرار'])
        }
        vectors.append({'id': str(idx), 'values': vector, 'metadata': metadata})
        time.sleep(1)  # Pause d'une seconde entre les requêtes pour éviter les limitations de taux
    except cohere.error.CohereAPIError as e:
        print(f"Erreur lors de l'encodage du texte à l'index {idx}: {e}")
        break

# Fonction pour diviser les vecteurs en lots
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Taille de lot recommandée pour rester en dessous de 4 Mo
batch_size = 100

# Uploader les vecteurs dans Pinecone par lots
for batch in chunks(vectors, batch_size):
    index.upsert(vectors=batch)
    print(f"Lot de {len(batch)} vecteurs uploadé avec succès dans Pinecone")

print("Toutes les données ont été uploadées avec succès dans Pinecone")

Lot de 100 vecteurs uploadé avec succès dans Pinecone
Lot de 100 vecteurs uploadé avec succès dans Pinecone
Lot de 100 vecteurs uploadé avec succès dans Pinecone
Lot de 79 vecteurs uploadé avec succès dans Pinecone
Toutes les données ont été uploadées avec succès dans Pinecone
